In [1]:
import numpy as np
from scipy.io import loadmat
import src
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import scipy

In [2]:
def load_keypoints(file_path):
    data = loadmat(file_path)
    kp = data['kp']  #Keypoints (Nx2 matrix)
    desc = data['desc']  # Descritores(NxD matrix)
    return kp, desc

In [3]:
# Inicializar o mosaico
initial_image_path = 'ISRwall/input_1/images/img_0001.jpg'
initial_image = src.image_to_matrix(initial_image_path)
width, height = (20000,10000)
start=2000
dst = np.full((height, width, initial_image.shape[2] if initial_image.ndim == 3 else 1), 0, dtype=initial_image.dtype)

for y in range(initial_image.shape[0]):
    for x in range(initial_image.shape[1]):
        if 0 <= x+12000 < width and 0 <= y+6000 < height:
            dst[y+6000, x+12000] = initial_image[y, x]

src.matrix_to_image(dst, f"final_mosaic.jpg")

In [4]:
counter = 0

kp1, desc1 = load_keypoints(f"ISRwall/input_1/keypoints/kp_000{counter+1}.mat")
kp2, desc2 = load_keypoints(f"ISRwall/input_1/keypoints/kp_000{counter+2}.mat")



print("kp1: ", kp1.shape)
print("desc1: ", desc1.shape)
print("kp2: ", kp2.shape)
print("desc2: ", desc2.shape)

kp1:  (6106, 2)
desc1:  (6106, 64)
kp2:  (6808, 2)
desc2:  (6808, 64)


In [5]:
def matching_optional(desc1, desc2, threshold=0.25):
    
    distances = scipy.spatial.distance.cdist(desc1, desc2, 'euclidean') ## calculate the distances between each query descriptor and each train descriptor: i -> query, j -> train
    sorted_indices = np.argsort(distances, axis=1)[:, :2]
    
    first_min_indices = sorted_indices[:, 0]
    second_min_indices = sorted_indices[:, 1]

    first_min_distances = distances[np.arange(distances.shape[0]), first_min_indices]
    second_min_distances = distances[np.arange(distances.shape[0]), second_min_indices]

    condition = first_min_distances < threshold * second_min_distances
    matches = np.column_stack((np.where(condition)[0], first_min_indices[condition]))
    
    return matches

In [7]:
matches = matching_optional(desc1, desc2)

In [12]:
inlier_threshold = 2.0
batch_size = 2

batch_sample = np.random.choice(matches_2.shape[0], (batch_size, 4), replace=False)

src_pts_batches = kp1[matches_2[batch_sample[:, 0]]]
dts_pts_batches = kp2[matches_2[batch_sample[:, 1]]]

H_batches = np.array([src.getPerspectiveTransform(src_pts, dst_pts) for src_pts, dst_pts in zip(src_pts_batches, dts_pts_batches)])

H_batches_unified = np.hstack(H_batches)

In [14]:
print(H_batches)
print(H_batches.shape)

print()
print(H_batches_unified)
print(H_batches_unified.shape)

[[[ 9.14227920e-05 -5.00520468e-04 -1.13462550e-04]
  [-7.74837042e-06  3.02409560e-06  1.08587197e-08]
  [-4.80541678e-04 -2.02083371e-04  1.00000000e+00]]

 [[ 3.19382433e-04  5.23468434e-05 -3.08288390e-04]
  [ 3.42696349e-04 -2.45860916e-04 -1.41544740e-08]
  [-6.02802379e-04 -1.19401847e-04  1.00000000e+00]]]
(2, 3, 3)

[[ 9.14227920e-05 -5.00520468e-04 -1.13462550e-04  3.19382433e-04
   5.23468434e-05 -3.08288390e-04]
 [-7.74837042e-06  3.02409560e-06  1.08587197e-08  3.42696349e-04
  -2.45860916e-04 -1.41544740e-08]
 [-4.80541678e-04 -2.02083371e-04  1.00000000e+00 -6.02802379e-04
  -1.19401847e-04  1.00000000e+00]]
(3, 6)


In [21]:
src_pts_hom_aux = np.column_stack((kp1[matches_2[:, 0]], np.ones((matches_2.shape[0], 1))))
print(src_pts_hom_aux[:10])

[[1.52867529e+03 1.10878992e+03 1.00000000e+00]
 [2.80704980e+03 1.06965356e+03 1.00000000e+00]
 [1.51490808e+03 8.26793152e+02 1.00000000e+00]
 [2.11259399e+03 8.11127136e+02 1.00000000e+00]
 [2.19201953e+03 8.09396790e+02 1.00000000e+00]
 [2.65929443e+03 1.12781799e+03 1.00000000e+00]
 [1.98127258e+03 8.35782349e+02 1.00000000e+00]
 [1.77079089e+03 1.28695178e+03 1.00000000e+00]
 [2.13739819e+03 1.35662537e+03 1.00000000e+00]
 [2.15568701e+03 1.31542883e+03 1.00000000e+00]]


In [20]:
src_pts_hom_zero = np.column_stack((kp1[matches_2[:, 0]], np.ones((matches_2.shape[0], 1)))).T
src_pts_hom_batch = np.vstack()

(3, 170)
[[1.52867529e+03 2.80704980e+03 1.51490808e+03 2.11259399e+03
  2.19201953e+03]
 [1.10878992e+03 1.06965356e+03 8.26793152e+02 8.11127136e+02
  8.09396790e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00]]
